In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mpl_toolkits.axisartist as axisartist
from tqdm import tqdm
from numba import jit

In [17]:
fake_goal_coords = (0, 0, 0)  # 假目标坐标
real_goal_coords = (0, 200, 0)  # 真目标坐标
M1_coords = (20000, 0, 2000)  # 导弹 M1 坐标
M2_coords = (19000, 600, 2100)  # 导弹 M2 坐标
M3_coords = (18000, -600, 1900)  # 导弹 M3 坐标
FY1_coords = (17800, 0, 1800)  # 无人机 FY1 坐标
FY2_coords = (12000, 1400, 1400)  # 无人机 FY2 坐标
FY3_coords = (6000, -3000, 700)  # 无人机 FY3 坐标
FY4_coords = (11000, 2000, 1800)  # 无人机 FY4 坐标
FY5_coords = (13000, -2000, 1300)  # 无人机 FY5 坐标

问题1无需考虑 $y$ 轴，只考虑 $x$ 轴和 $z$ 轴。时间从警戒雷达发现来袭导弹时(同时无人机受领任务)起算，此外还有无人机投放烟幕干扰弹、烟幕干扰弹起爆(同时不一定为目标提供有效遮蔽)、烟幕干扰弹无法为目标提供有效遮蔽、导弹到达假目标一共11个时刻，分别定义为 $t_0, t_{1,1}, t_{2,1}, t_{3,1}, t_{1,2}, t_{2,2}, t_{3,3}, t_{1,3}, t_{2,3}, t_{3,2}, t_4$

**导弹坐标-时间函数**：

$$\begin{cases}
x_{\text{M}_1}(t) = x_{\text{M}_1,o} - v_{\text{M}_1} \frac{x_{\text{M}_1,o}}{\sqrt{x_{\text{M}_1,o}^2 + z_{\text{M}_1,o}^2}} t \\
y_{\text{M}_1}(t) = y_{\text{M}_1,o} = 0 \\
z_{\text{M}_1}(t) = z_{\text{M}_1,o} - v_{\text{M}_1} \frac{z_{\text{M}_1,o}}{\sqrt{x_{\text{M}_1,o}^2 + z_{\text{M}_1,o}^2}} t \\
\end{cases}$$

其中，$(x_{\text{M}_1,o}, y_{\text{M}_1,o}, z_{\text{M}_1,o})$ 为导弹初始坐标，$v_{\text{M}_1} = 300 \text{m/s}$ 为导弹速度，方向在 $xyz$ 坐标系为 $\overrightarrow{P_{\text{M}_1}P_{\text{G}_f}}$

**无人机Fi坐标-时间函数**：

$$\begin{cases}
x_{\text{F}_i}(t) = x_{\text{F}_i,o} + v_{\text{F}_i} \cos \theta_{\text{F}_i} t \\
y_{\text{F}_i}(t) = y_{\text{F}_i,o} + v_{\text{F}_i} \sin \theta_{\text{F}_i} t \\
z_{\text{F}_i}(t) = z_{\text{F}_i,o} \\
\end{cases}, t \le t_{1,i}, i \in \{1, 2, 3\}$$

其中，$(x_{\text{F}_i,o}, y_{\text{F}_i,o}, z_{\text{F}_i,o})$ 为无人机初始坐标，$\theta_{\text{F}_i}, v_{\text{F}_i}$ 为无人机飞行方向和速度

**烟幕干扰弹坐标-时间函数**：

$$\begin{cases}
x_{\text{B}_i}(t) = x_{\text{B}_i,o} + v_{\text{F}_1} \cos \theta_{\text{F}_1} (t - t_{1,i}) \\
y_{\text{B}_i}(t) = y_{\text{B}_i,o} + v_{\text{F}_1} \sin \theta_{\text{F}_1} (t - t_{1,i}) \\
z_{\text{B}_i}(t) = z_{\text{B}_i,o} - \frac{1}{2} g (t - t_{1,i})^2 \\
\end{cases}, t_{1,i} < t \le t_{2,i}, i \in \{1, 2, 3\}$$

其中，$(x_{\text{B}_i,o}, y_{\text{B}_i,o}, z_{\text{B}_i,o})$ 为第 $i$ 个无人机的烟幕干扰弹初始坐标

**烟幕云团坐标-时间函数**：

$$\begin{cases}
x_{\text{C}_i}(t) = x_{\text{C}_i,o} \\
y_{\text{C}_i}(t) = y_{\text{C}_i,o} \\
z_{\text{C}_i}(t) = z_{\text{C}_i,o} - v_{\text{C}} (t - t_{2,i}) \\
\end{cases}, t_{2,i} < t \le t_{3,i}, i \in \{1, 2, 3\}$$

其中，$(x_{\text{C}_i,o}, y_{\text{C}_i,o}, z_{\text{C}_i,o})$ 为第 $i$ 个无人机的烟幕云团初始坐标，$v_{\text{C}} = 3 \text{m/s}$ 为烟幕云团下沉速度

目标函数：

$$\max_{\theta_{F_i}, v_{F_i}, t_{m,i}, t_{e,i}} \int_{t_{s_0}}^{t_{s_1}} \text{is\_shield}(t) dt \\
\text{s.t.} \begin{cases}
\text{is\_shield}(t) = \begin{cases} 1 & \text{if} ~ \max(|P_{M_i} P_C|) < r_C \\ 0 &  \text{otherwise} \end{cases} \\
t_0 + t_{m,i} + t_{e,i} + 20 = t_{1,i} + t_{e,i} + 20 = t_{2,i} + 20 = t_{3,i} < t_4 \\
\theta_{F_1} \in [0, 2 \pi) \\
v_{F_1} \in [70, 140] \\
t_{m,1}, t_{m,2}, t_{m,3} \in [0, 5] \\
t_{e,1}, t_{e,2}, t_{e,3} \in [0, 5] \\
i \in \{1, 2, 3\}
\end{cases}$$

其中，$\text{is\_shield}(t)$ 表示在 $t$ 时刻导弹是否被烟幕云团遮蔽，$t_{m,i}, t_{e,i}$ 分别表示无人机受领任务与投放烟幕干扰弹的时间间隔、烟幕干扰弹投放与起爆时间间隔